In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
from hvplot import pandas
import string

#############Account number assignment definitions#################
def account(name):
    if name == "loan_balance":
        return 111111
    elif name == "interest_receivable":
        return 444444
    elif name == 'collection_fees':
        return 800001
    elif name =='loan_trailing_fee':
        return 800002
    elif name == 'general_receivable':
        return 222222
    else:
        print("account not found")
        
##################IMPORT LOAN DATASET ###############        

loan_data = Path("../fintech_collab/loan_data.xlsx")

loan_df = pd.read_excel(loan_data, header=0, index_col="Loan ID")

######################IMPORT CATEGORY DATASET #######################
ups_hfs_data = Path("../fintech_collab/category.xlsx")
ups_hfs_df =pd.read_excel(ups_hfs_data, header=0)

Left_join = pd.merge(loan_df, ups_hfs_df, on = 'Loan ID', how='left')
Left_join['Category_y'] = Left_join['Category_y'].fillna('HFI')

##########################Cleaning Data#################################

#loan_df['Purchase_Date'] = (datetime.utcfromtimestamp(0) + timedelta(loan_df['Purchase Date']).strftime("%Y-%m-%d"))

loan_df.columns = loan_df.columns.str.strip()
loan_df.columns =loan_df.columns.str.replace(' ','_')

# fm_name = list(map(chr, range(97, 114)))
#can list be at cutting list of loan_df.columns[:3]
############ create a list from the columns of Loan Data ###############################
listA = ['Loan_Amount',
       'Beginning_Period_Principal_Outstanding',
       'Ending_Principal_Outstanding', 'Beginning_Period_Accrued_Interest',
       'Ending_Period_Accrued_Interest', 'Principal_Paid', 'Interest_Paid',
       'Late_Fee_Paid', 'Return_Item_Fee_Paid', 'Overpayment', 'Sundry_Loss',
       'Total_Payments_Received', 'Collection_Fees', 'Loan_Trailing_Fee',
       'Onboarding_Loans', 'Charge_offs']

print("Aggregate sum of each accounts is the following: \n")
# fm_name = list(map(chr, range(97, 114)))
# loop through the list to get total sum for each columns
j = 0

for i in listA:    
    i = loan_df[f"{i}"].sum()
    print(f" {listA[j]} is: {round(i,2):,}")
    listA[j] = round(i,2)
    j= j+1
    

###################### VALIDATE ENDING PRINCIPAL OUTSTANDING BALANCE ####################
#begining + onboarding loans - principal paid - charge off
cal_ending = listA[1]+ listA[14] - listA[5] - listA[-1]
print(f"\n Calculated ending principal outstanding is {cal_ending:,}\n")

print(f"Journal entry form is below:\n")

######### CREATE TABLE for PRINCIPAL PAYMENT ########################
prin_pmt = listA[5]
if prin_pmt >= 0:
    pmt_entry_df = pd.DataFrame({
    'AccountNumber': ['222222', account('loan_balance')],
    'Amount':[prin_pmt, -prin_pmt],
    'Description':['Principal Payment'],
    'AccountName':['General Receivable', 'Loan Balance']},
    index = ['dr', 'cr'])
    
else:
    pmt_entry_df = pd.DataFrame({
    'AccountNumber': ['222222', account('loan_balance')],
    'Amount':[-prin_pmt, prin_pmt],
    'Description':['Principal Payment'],
    'AccountName':['General Receivable', 'Loan Balance']},
    index = ['dr', 'cr'])
    
pmt_entry_df.head()

###############CREATE TABLE FOR CHARGE OFFS###############################
charge_offs = listA[15]
if charge_offs >= 0:
    ch_entry_df= pd.DataFrame({
    'AccountNumber': ['222222', account('loan_balance')],
    'Amount':[charge_offs, -charge_offs],
    'Description':['Charge offs', 'Charge offs'],
    'AccountName':['General Receivable', 'Loan Balance']},
    index = ['dr', 'cr'])
else:
    ch_entry_df = pd.DataFrame({
    'AccountNumber': ['222222', account('loan_balance')],
    'Amount':[-charge_offs, charge_offs],
    'Description':['Charge offs', 'Charge offs'],
    'AccountName':['General Receivable', 'Loan Balance']},
    index = ['dr', 'cr'])
    
###########CREATE TABLE FOR INTEREST INCOME ENTRY ##################

interest_accrued = listA[4] - listA[3]
if interest_accrued >=0:
    int_entry_df= pd.DataFrame({
        'AccountNumber': [account('interest_receivable'),'555555'],
        'Amount':[interest_accrued, -interest_accrued],
        'Description':['Change in interest accrual'],
        'AccountName':['Interest Receivable', 'Interest Income']},
        index = ['dr', 'cr'])
else:
    int_entry_df = pd.DataFrame({
    'AccountNumber': [account('interest_receivable'),'555555'],
    'Amount':[-interest_accrued, interest_accrued],
    'Description':['Change in interest accrual'],
    'AccountName':['Interest Receivable', 'Interest Income']},
    index = ['dr', 'cr'])


###########CREATE TABLE FOR INTEREST payment ENTRY ##################

interest_pmt = prin_pmt = listA[5]
if interest_accrued >=0:
    int_pmt_df= pd.DataFrame({
        'AccountNumber': [account('general_receivable'),'555555'],
        'Amount':[interest_pmt, -interest_pmt],
        'AccountName':['General Receivable', 'Interest Income'],
        'Description':['interest payment']},
        index = ['dr', 'cr'])
else:
    int_pmt_df = pd.DataFrame({
    'AccountNumber': [account('general_receivable'),'555555'],
    'AccountName':['General Receivable', 'Interest Income'],
    'Amount':[-interest_pmt, interest_pmt],
    'Description':['interest payment']},
    index = ['dr', 'cr'])

###########CREATE TABLE FOR Service fees ##################
service_fees = listA[12] + listA[13]

if service_fees >=0:
    ser_fee_df= pd.DataFrame({
        'AccountNumber': ['777777', account('general_receivable')],
        'Amount':[service_fees, -service_fees],
        'Description':['Service Fee expense'],
        'AccountName':['Service fee expense', 'General Receivable']},
        index = ['dr', 'cr'])
else:
    ser_fee_df = pd.DataFrame({
    'AccountNumber': ['777777', account('general_receivable')],
    'Amount':[-service_fees, service_fees],
    'Description':['Service Fee expense'],
    'AccountName':['Service fee expense', 'General Receivable']},
    index = ['dr', 'cr'])

###### COMBINED TABLES ###################################################3
int_entry_combined = pd.concat([ pmt_entry_df, int_entry_df, int_pmt_df, ch_entry_df, ser_fee_df ], axis=0 )
int_entry_combined.head(10)

Aggregate sum of each accounts is the following: 

 Loan_Amount is: 61,795,483
 Beginning_Period_Principal_Outstanding is: 47,927,569.16
 Ending_Principal_Outstanding is: 57,185,195.94
 Beginning_Period_Accrued_Interest is: 192,109.65
 Ending_Period_Accrued_Interest is: 307,101.75
 Principal_Paid is: 1,546,876.11
 Interest_Paid is: 447,726.19
 Late_Fee_Paid is: 8.63
 Return_Item_Fee_Paid is: 130
 Overpayment is: 1,156.56
 Sundry_Loss is: 0
 Total_Payments_Received is: 1,995,897.49
 Collection_Fees is: 1,467.12
 Loan_Trailing_Fee is: 3.61
 Onboarding_Loans is: 10,814,398.07
 Charge_offs is: 9,895.18

 Calculated ending principal outstanding is 57,185,195.94

Journal entry form is below:



,AccountNumber,Amount,Description,AccountName
dr,222222,1546876.11,Principal Payment,General Receivable
cr,111111,-1546876.11,Principal Payment,Loan Balance
dr,444444,114992.10,Change in interest accrual,Interest Receivable
cr,555555,-114992.10,Change in interest accrual,Interest Income
dr,222222,1546876.11,interest payment,General Receivable
cr,555555,-1546876.11,interest payment,Interest Income
dr,222222,9895.18,Charge offs,General Receivable
cr,111111,-9895.18,Charge offs,Loan Balance
dr,777777,1470.73,Service Fee expense,Service fee expense
cr,222222,-1470.73,Service Fee expense,General Receivable
